# Named Entity Recognition Using BERT
## Summary
This notebook demonstrates how to fine tune [pretrained BERT model](https://github.com/huggingface/pytorch-pretrained-BERT) for named entity recognition (NER) task. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, and model evaluation. 

[BERT (Bidirectional Transformers for Language Understanding)](https://arxiv.org/pdf/1810.04805.pdf) is a powerful pre-trained lanaguage model that can be used for multiple NLP tasks, including text classification, question answering, named entity recognition, etc. It's able to achieve state of the art performance with only a few epochs of fine tuning on task specific datasets.  
The figure below illustrates how BERT can be fine tuned for NER tasks. The input data is a list of tokens representing a sentence. In the training data, each token has an entity label. After fine tuning, the model predicts an entity label for each token in a given testing sentence. 

<img src="https://nlpbp.blob.core.windows.net/images/bert_architecture.png">

## Required packages
* pytorch
* pytorch-pretrained-bert
* pandas
* seqeval

In [1]:
import sys
import os
import random
from seqeval.metrics import f1_score, classification_report

import torch
from pytorch_pretrained_bert.tokenization import BertTokenizer

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.bert.token_classification import BERTTokenClassifier, postprocess_token_labels
from utils_nlp.bert.common import Language, Tokenizer
from utils_nlp.dataset.wikigold import download, read_data, get_train_test_data, get_unique_labels

## Configurations

In [2]:
# path configurations
data_dir = "./data"
data_file = "./data/wikigold.conll.txt"
cache_dir="."

# set random seeds
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)

# model configurations
language = Language.ENGLISHCASED
do_lower_case = False
max_seq_length = 200

# training configurations
device="gpu"
batch_size = 16
num_train_epochs = 5

# optimizer configuration
learning_rate = 3e-5

## Preprocess Data

### Get training and testing data
The dataset used in this notebook is the [wikigold dataset](https://www.aclweb.org/anthology/W09-3302). The wikigold dataset consists of 145 mannually labelled Wikipedia articles, including 1841 sentences and 40k tokens in total. The dataset can be directly downloaded from [here](https://github.com/juand-r/entity-recognition-datasets/tree/master/data/wikigold). The `download` function downloads the data file to a user-specified directory.  

The helper function `get_train_test_data` splits the dataset into training and testing sets according to `test_percentage`. Because this is a relatively small dataset, we set `test_percentage` to 0.5 in order to have enough data for model evaluation. Running this notebook multiple times with different random seeds produces similar results.   

The helper function `get_unique_labels` returns the unique entity labels in the dataset. There are 5 unique labels in the   original dataset: 'O' (non-entity), 'I-LOC' (location), 'I-MISC' (miscellaneous), 'I-PER' (person), and 'I-ORG' (organization). An 'X' label is added for the trailing word pieces generated by BERT, because BERT uses WordPiece tokenizer.  

The maximum number of words in a sentence is 144, so we set `max_seq_length` to 200 above, because the number of tokens will grow after WordPiece tokenization.

In [3]:
download(data_dir)
wikigold_text = read_data(data_file)
train_text, train_labels, test_text, test_labels = get_train_test_data(wikigold_text, test_percentage=0.5)
label_list = get_unique_labels()
print('\nUnique entity labels: \n{}\n'.format(label_list))
print('Sample sentence: \n{}\n'.format(train_text[0]))
print('Sample sentence labels: \n{}\n'.format(train_labels[0]))

Maximum sequence length in training data is: 144
Maximum sequence length in testing data is: 89

Unique entity labels: 
['O', 'I-LOC', 'I-MISC', 'I-PER', 'I-ORG', 'X']

Sample sentence: 
-DOCSTART-

Sample sentence labels: 
['O']



### Tokenization and Preprocessing
The `preprocess_ner_tokens` method of the `Tokenizer` class converts raw string data to numerical features, involving the following steps:
1. WordPiece tokenization.
2. Convert tokens and labels to numerical values, i.e. token ids and label ids.
3. Sequence padding or truncation according to the `max_seq_length` configuration.

**Create a dictionary that maps labels to numerical values**

In [4]:
label_map = {label: i for i, label in enumerate(label_list)}

**Create a tokenizer**

In [5]:
tokenizer = Tokenizer(language=language, 
                      to_lower=do_lower_case, 
                      cache_dir=cache_dir)

**Create numerical features**  
Note there is an argument called `trailing_piece_tag`. BERT uses a WordPiece tokenizer which breaks down some words into multiple tokens, e.g. "playing" is tokenized into "play" and "##ing". Since the input data only come with one token label for "playing", within `prerocess_ner_tokens`, the original token label is assigned to the first token "play" and the second token "##ing" is labeled as "X". By default, `trailing_piece_tag` is set to "X". If "X" already exists in your data, you can set `trailing_piece_tag` to another value that doesn't exist in your data. 

In [6]:
train_token_ids, train_input_mask, train_trailing_token_mask, train_label_ids = \
    tokenizer.preprocess_ner_tokens(text=train_text,
                                    label_map=label_map,
                                    max_len=max_seq_length,
                                    labels=train_labels,
                                    trailing_piece_tag="X")
test_token_ids, test_input_mask, test_trailing_token_mask, test_label_ids = \
    tokenizer.preprocess_ner_tokens(text=test_text,
                                    label_map=label_map,
                                    max_len=max_seq_length,
                                    labels=test_labels,
                                    trailing_piece_tag="X")

`Tokenizer.preprocess_ner_tokens` outputs three or four lists of numerical features lists, each sublist contains features of an input sentence: 
1. token ids: list of numerical values each corresponds to a token.
2. attention mask: list of 1s and 0s, 1 for input tokens and 0 for padded tokens, so that padded tokens are not attended to. 
3. trailing word piece mask: boolean list, `True` for the first word piece of each original word, `False` for the trailing word pieces, e.g. ##ing. This mask is useful for removing predictions on trailing word pieces, so that each original word in the input text has a unique predicted label. 
4. label ids: list of numerical values each corresponds to an entity label, if `labels` is provided.

In [7]:
print("Sample token ids:\n{}\n".format(train_token_ids[0]))
print("Sample attention mask:\n{}\n".format(train_input_mask[0]))
print("Sample trailing token mask:\n{}\n".format(train_trailing_token_mask[0]))
print("Sample label ids:\n{}\n".format(train_label_ids[0]))

Sample token ids:
[118, 141, 9244, 9272, 12426, 1942, 118, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

## Create Token Classifier
The value of the `language` argument determines which BERT model is used:
* Language.ENGLISH: "bert-base-uncased"
* Language.ENGLISHCASED: "bert-base-cased"
* Language.ENGLISHLARGE: "bert-large-uncased"
* Language.ENGLISHLARGECASED: "bert-large-cased"
* Language.CHINESE: "bert-base-chinese"
* Language.MULTILINGUAL: "bert-base-multilingual-cased"

Here we use the base, uncased pretrained model.

In [8]:
token_classifier = BERTTokenClassifier(language=language,
                                       num_labels=len(label_list),
                                       cache_dir=cache_dir)

## Train Model

In [9]:
token_classifier.fit(token_ids=train_token_ids, 
                     input_mask=train_input_mask, 
                     labels=train_label_ids,
                     num_epochs=num_train_epochs, 
                     batch_size=batch_size, 
                     learning_rate=learning_rate)

t_total value of -1 results in schedule not being applied
Iteration:   0%|          | 0/58 [00:00<?, ?it/s]


Iteration:   0%|          | 0/58 [00:00<?, ?it/s]

Train loss: 0.4810219132180872



Iteration:   0%|          | 0/58 [00:00<?, ?it/s]

Train loss: 0.1079183994182225


...

Train loss: 0.04469434472186298



Iteration:   0%|          | 0/58 [00:00<?, ?it/s]

Train loss: 0.023166427043555624



Epoch: 100%|██████████| 5/5 [06:18<00:00, 75.79s/it]2s/it]

Train loss: 0.014027320994626218


## Predict on Test Data

In [10]:
pred_label_ids = token_classifier.predict(token_ids=test_token_ids, 
                                          input_mask=test_input_mask, 
                                          labels=test_label_ids, 
                                          batch_size=batch_size)

Iteration:   0%|          | 0/58 [00:00<?, ?it/s]

Iteration: 100%|██████████| 58/58 [00:25<00:00,  2.30it/s]

Evaluation loss: 0.13430038492741256


## Evaluate Model
The `predict` method of the token classifier outputs label ids for all tokens, including the padded tokens. `postprocess_token_labels` is a helper function that removes the predictions on padded tokens. If a `label_map` is provided, it maps the numerical label ids back to original token labels which are usually string type. 

In [11]:
pred_tags_no_padding = postprocess_token_labels(pred_label_ids, 
                                                test_input_mask, 
                                                label_map)
true_tags_no_padding = postprocess_token_labels(test_label_ids, 
                                                test_input_mask, 
                                                label_map)
print(classification_report(true_tags_no_padding, pred_tags_no_padding, digits=2))

           precision    recall  f1-score   support

      LOC       0.91      0.84      0.88       558
        X       0.97      0.98      0.98      2060
      PER       0.90      0.95      0.93       569
      ORG       0.70      0.82      0.76       553
     MISC       0.74      0.71      0.72       399

micro avg       0.89      0.91      0.90      4139
macro avg       0.90      0.91      0.90      4139



`postprocess_token_labels` also provides an option to remove the predictions on trailing word pieces, e.g. ##ing, so that the final predicted labels correspond to the original words in the input text. The `trailing_token_mask` is obtained from `tokenizer.preprocess_ner_tokens`

In [12]:
pred_tags_no_padding_no_trailing = postprocess_token_labels(pred_label_ids, 
                                                            test_input_mask, 
                                                            label_map, 
                                                            remove_trailing_word_pieces=True, 
                                                            trailing_token_mask=test_trailing_token_mask)
true_tags_no_padding_no_trailing = postprocess_token_labels(test_label_ids, 
                                                            test_input_mask, 
                                                            label_map, 
                                                            remove_trailing_word_pieces=True, 
                                                            trailing_token_mask=test_trailing_token_mask)
print(classification_report(true_tags_no_padding_no_trailing, pred_tags_no_padding_no_trailing, digits=2))

           precision    recall  f1-score   support

      LOC       0.90      0.85      0.88       507
      PER       0.89      0.94      0.92       460
      ORG       0.65      0.81      0.72       440
     MISC       0.67      0.68      0.68       328

micro avg       0.77      0.83      0.80      1735
macro avg       0.79      0.83      0.81      1735



We can see that the metrics are worse after excluding trailing word pieces, because they are easy to predict. 